## SILVER TO GOLD LAYER

### Gold Layer - Profit and Loss


In [ ]:
# Importing Common Utility Function
import pandas as pd

from StockETL import GlobalPath, Portfolio

In [ ]:
# Import necessary libraries and utility functions
%run ../COMMON/common_utility.ipynb

In [ ]:
# Instantiate GlobalPath
tradehistory_silver_file_path = GlobalPath(
    f"DATA/SILVER/TradeHistory/TradeHistory_data.csv"
)
profitloss_gold_file_path = GlobalPath(
    f"DATA/GOLD/ProfitLoss/ProfitLoss_data.csv"
)
profitloss_gold_schema_file_path = GlobalPath(
    "CONFIG/DATA_CONTRACTS/GOLD/ProfitLoss.json"
)

### Data Processing

- Read and sort trade history data.
- Apply portfolio trade logic.


In [ ]:
# Read the CSV file
df_trade_history = pd.read_csv(tradehistory_silver_file_path)

# Convert 'datetime' to datetime type
df_trade_history["datetime"] = pd.to_datetime(df_trade_history["datetime"])

# Sort the DataFrame by 'datetime'
df_trade_history = df_trade_history.sort_values(by="datetime")

print(f"Read SILVER Layer trade history data from => {tradehistory_silver_file_path}")

### Portfolio Logic Application

- Instantiate Portfolio and apply trade logic.
- Handle expired stocks


In [ ]:
# Apply the trade logic to each row of the DataFrame
# Instantiate the Portfolio object
portfolio = Portfolio()
for record in df_trade_history.to_dict(orient="records"):
    portfolio.trade(record)

portfolio.check_expired_stocks()
# expired_stocks example : df_trade_history["scrip_name"] == "NIFTY-PE-24650-18JUL2024"

### Final Processing and Export

- Select and sort relevant columns.
- Save the processed data as a CSV file in the Gold layer.


In [ ]:
# Create a DataFrame from the processed data
df_pnl = pd.DataFrame(portfolio.get_pnl())

# Reset index to ensure it starts from 0
df_pnl = df_pnl.reset_index(drop=True)

# Align Datafame with DataContract
df_pnl = align_with_datacontract(df_pnl, profitloss_gold_schema_file_path)

# Save the final DataFrame to a CSV file
df_pnl.to_csv(profitloss_gold_file_path, index=None)

print("GOLD Layer CSV file for ProfitLoss successfully created at =>")
print(profitloss_gold_file_path)

# Display the DataFrame debugrmation
df_pnl.info()